# Feature Extraction Functions

In [17]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go
import numpy as np

In [18]:
def get_reversal(points,rate = 0,min_period = 0):
  change = 0
  reversal_1 = 0
  reversal_2 = 0
  reversal_1_index = 0
  reversal_2_index = 0
  mins = []
  maxs = []
  mins_rates = []
  maxs_rates = []
  last = "none"
  last_index = 0
  if not len(points) in [0,1]:
    for i in range(1,len(points)-1):
          old_change = change
          if points[i+1] >  points[i]:
            change = 1
          elif points[i+1] <  points[i]:
            change = -1
          if old_change != change:
            if reversal_1 == 0:
                reversal_1 = points[i]
                reversal_1_index = i
            else:
                reversal_2 = points[i]
                reversal_2_index = i
                r = (reversal_2 - reversal_1) / reversal_1
                if abs(r) >= rate:
                  if change == 1:
                    if last == "none" or last == "min":
                      if i - last_index > min_period:
                        maxs.append(reversal_1_index)
                        maxs_rates.append(r*100)
                        last = "max"
                        last_index = i
                  else:
                    if last == "none" or last == "max":
                      if i - last_index > min_period:
                        mins.append(reversal_1_index)
                        mins_rates.append(r*100)
                        last = "min"
                        last_index = i
                reversal_1 = reversal_2
                reversal_1_index = reversal_2_index
    if change == 1:
          mins.append(len(points)-2)
    else:
          maxs.append(len(points)-2)
  return mins, maxs, mins_rates, maxs_rates

def get_mins(poitns,maxs):
    mins = []
    for i in range(len(maxs)):
      if i == 0:
          start = 0
          end = maxs[i]
      else:
          start = maxs[i-1]
          end = maxs[i]

      idxmin_loc = poitns.iloc[start+1:end].idxmin()
      idx_loc = poitns.index.get_loc(idxmin_loc)
      mins.append(idx_loc)
    return mins

def get_reversal_rates(points,maxs,mins):
   indexes = mins + maxs
   indexes.sort()
   change_rates = []
   for i in range(1, len(indexes)):
      prev_index = indexes[i-1]
      curr_index = indexes[i]
      prev_val = points[prev_index]
      curr_val = points[curr_index]
      change_rate = (curr_val - prev_val) / prev_val * 100
      change_rates.append(change_rate)
   return change_rates


def add_reversal(df,points,mins, maxs):
  df["Reversal"] = 0
  df["Reversal"][mins] = 1
  df["Reversal"][maxs] = 2
  df["Reversal_rate"] = 0
  l = mins+maxs
  l.sort()
  df["Reversal_rate"].iloc[l[:-1]] =  get_reversal_rates(points,maxs,mins)
  df["next_Reversal"] = df["Reversal"].shift(-1, axis = 0).fillna(0)

def add_major_trend(df,mins,maxs):
  df['Major_trend'] = 0
  idx_list = mins + maxs
  idx_list.sort()
  if min(mins) < min(maxs):
    trend = -1
  else:
    trend = 1
  for i in range(len(idx_list)):
      if i == 0:
          start = 0
          end = idx_list[i]
      else:
          start = idx_list[i-1]
          end = idx_list[i]
      df.loc[start:end,'Major_trend'] = trend
      trend *= -1
  df.loc[idx_list[-1]:,'Major_trend'] = trend

def add_minor_trend(df,mins,maxs):
  df['Minor_trend'] = 0
  idx_list = mins + maxs
  idx_list.sort()
  if min(mins) < min(maxs):
    trend = -1
  else:
    trend = 1
  for i in range(len(idx_list)):
      if i == 0:
          start = 0
          end = idx_list[i]
      else:
          start = idx_list[i-1]
          end = idx_list[i]
      df.loc[start:end,'Minor_trend'] = trend
      trend *= -1
  df.loc[idx_list[-1]:,'Minor_trend'] = trend

def add_trend_age(df,mins,maxs):
  idx_list = mins + maxs
  idx_list.sort()
  ret = []
  minus = 0
  for i in range(len(df)):
    if i-1 in idx_list:
      minus = i
    ret.append(i-minus+1)
  df["trend_age"] = ret



In [19]:
# Support Resistance
def isFarFromLevel(l,s,levels):
   return np.sum([abs(l-x) < s  for x in levels.Value.values]) == 0

def get_levels(max_maxs, min_mins, high_points,low_points,df):
  lvls = pd.DataFrame(columns = ['Start', 'Value'])
  s =  np.mean(df['High'] - df['Low'])
  indexes = max_maxs+min_mins
  indexes.sort()
  for i in indexes:
    if i in max_maxs:
      l = high_points[i]
      if isFarFromLevel(l,s,lvls):
          lvls.loc[len(lvls)] = [high_points.index[i], l]
    if i in min_mins:
      l = low_points[i]
      if isFarFromLevel(l,s,lvls):
          lvls.loc[len(lvls)] = [high_points.index[i], l]
  lvls = lvls.sort_values("Start").reset_index(drop = True)
  return lvls

lvls_features = ["age",'last_MReversal','nCandle','nCandleTime','nReversal','nMReversal','nReversalClose',"nReversalOpen","nReversalHigh","nReversalLow","nMReversalClose","nMReversalOpen","nMReversalHigh","nMReversalLow"]
def levels_propreties(df,lvls,analysis,current,reversal_dict):
    cut_lvls = lvls[lvls["Start"] < current]
    current_candle = df[df.index == current]
    for i in range(cut_lvls.shape[0]):
      l = cut_lvls.Value[i]
      s = cut_lvls.Start[i]
      analysis_index = analysis[analysis.Lvl == l].index
      # age
      age = (current - s).days
      analysis.loc[analysis_index,'age'] = age
      # last Mreversal age update
      analysis.loc[analysis_index,'last_MReversal'] += 1
      # nCandles
      zone = df.avg_candle[current] / 3
      touching = ((current_candle.Low - l < 0) & (current_candle.High - l > 0)) | ((current_candle.Low - l < zone) & (current_candle.Low - l > 0)) | ((l - current_candle.High < zone) & (current_candle.High - l < 0))
      touching = touching[touching.index[0]]
      if touching:
         analysis.loc[analysis_index,'nCandle'] += 1
         # last_MReversal
         if df[df.index == current].Reversal.iloc[0] == 1:
            analysis.loc[analysis_index,'last_MReversal'] = 0
         # nCandlesTime
         nCandle = analysis.loc[analysis_index,'nCandle']
         analysis.loc[analysis_index,'nCandleTime'] = nCandle / age
         # nReversal and nMreversal
         added = False
         Madded = False
         for ohlc_key in reversal_dict:
           for reversal_key in reversal_dict[ohlc_key]:
             if current in reversal_dict[ohlc_key][reversal_key]:
                analysis.loc[analysis_index,f"n{reversal_key}{ohlc_key}"] += 1
                if reversal_key == "Reversal":
                  if not added:
                    analysis.loc[analysis_index,"nReversal"] += 1
                    added = True
                else:
                  if not Madded:
                    analysis.loc[analysis_index,"nMReversal"] += 1
                    Madded = True
    return analysis

def add_SR(df):
   df["High-below"] = 0
   df["Low-below"] = 0
   df["Close-below"] = 0
   df["Open-below"] = 0
   df["Min-below"] = 0
   df["Max-below"] = 0
   df["above-High"] = 0
   df["above-Low"] = 0
   df["above-Close"] = 0
   df["above-Open"] = 0
   df["above-Min"] = 0
   df["above-Max"] = 0
   df['through-High'] = 0
   df['through-Low'] = 0
   df['through-Close'] = 0
   df['through-Open'] = 0
   df["through-Min"] = 0
   df["through-Max"] = 0

   df["no_below"] = 0
   df["no_above"] = 0
   df["no_through"] = 0

   for f in lvls_features:
     df[f"below_{f}"] = 0
     df[f"above_{f}"] = 0
     df[f"through_{f}"] = 0

  #  # number of lvls broken
  #  df["nBreakout"] = 0
  #  # strong level breakout
  #  df["nMBreakout"] = 0

   # get lvls
   high_points = df.High
   low_points = df.Low
   mins, maxs,r,r = get_reversal(high_points)
   max_points = high_points.loc[high_points.index[maxs]]
   max_mins, max_maxs,r,r = get_reversal(max_points)
   max_maxs = [maxs[i] for i in max_maxs]
   min_mins = get_mins(low_points,max_maxs)
   lvls = get_levels(max_maxs,min_mins,high_points,low_points,df)
   # get reversals and Mreversals
   reversal_dict = {"Close": {"Reversal":[],"MReversal":[]},"Open": {"Reversal":[],"MReversal":[]}
              ,"High": {"Reversal":[],"MReversal":[]},"Low": {"Reversal":[],"MReversal":[]}}
   OHLC = [df.Close,df.Open,df.High,df.Low]
   for points,key in zip(OHLC,reversal_dict):
    mins, maxs,r,r = get_reversal(points)
    reversal_dict[key]["Reversal"] = df.index[maxs+mins]
    max_mins, max_maxs,r,r = get_reversal(points.loc[points.index[maxs]])
    max_maxs = [maxs[i] for i in max_maxs]
    min_mins = get_mins(points,max_maxs)
    reversal_dict[key]["MReversal"] = df.index[max_maxs+min_mins]
   # initialize the propreties dataframe
   prop = pd.DataFrame(0, index=np.arange(len(lvls)),columns = ['Lvl','Start']+lvls_features)
   prop["Lvl"] = lvls["Value"]
   prop["Start"] = lvls["Start"]

   for i in range(df.shape[0]):
    current = df.index[i]
    prop = levels_propreties(df, lvls, prop, current,reversal_dict)
    cut_prop = prop[prop["Start"] < current]
    resistances = cut_prop.Lvl[cut_prop.Lvl - df.High[current] >= 0]
    resistance = None
    if len(resistances) > 0:
        resistance = cut_prop.iloc[resistances.idxmin()]
    supports = cut_prop.Lvl[cut_prop.Lvl - df.Low[current] <= 0]
    support = None
    if len(supports) > 0:
        support = cut_prop.iloc[supports.idxmax()]
    throughs = cut_prop.Lvl[(cut_prop.Lvl - df.Low[current] > 0) & (cut_prop.Lvl - df.High[current] < 0)]
    through = None
    if len(throughs) > 0:
        through = cut_prop.iloc[throughs.idxmax()] # i took the max just randomly
    Min = min(df.Open[current],df.Close[current])
    Max = max(df.Open[current],df.Close[current])
    if type(support) != type(None):
        df.loc[current, 'High-below'] = df.High[current] - support.Lvl
        df.loc[current, 'Low-below'] = df.Low[current] - support.Lvl
        df.loc[current, 'Close-below'] = df.Close[current] - support.Lvl
        df.loc[current, 'Open-below'] = df.Open[current] - support.Lvl
        df.loc[current, 'Min-below'] =  Min - support.Lvl
        df.loc[current, 'Max-below'] = Max - support.Lvl

        for val, column in zip(support[2:], support.index[2:]):
            df.loc[current, f'below_{column}'] = val
    else:
        df.loc[current, 'no_below'] = 1
    if type(resistance) != type(None):
        df.loc[current, 'above-High'] = resistance.Lvl - df.High[current]
        df.loc[current, 'above-Low'] = resistance.Lvl - df.Low[current]
        df.loc[current, 'above-Close'] = resistance.Lvl - df.Close[current]
        df.loc[current, 'above-Open'] = resistance.Lvl - df.Open[current]
        df.loc[current, 'above-Min'] = resistance.Lvl - Min
        df.loc[current, 'above-Max'] = resistance.Lvl - Max
        for val, column in zip(resistance[2:], resistance.index[2:]):
            df.loc[current, f'above_{column}'] = val
    else:
        df.loc[current, 'no_above'] = 1
    if type(through) != type(None):
        df.loc[current, 'through-High'] = through.Lvl - df.High[current]
        df.loc[current, 'through-Low'] = through.Lvl - df.Low[current]
        df.loc[current, 'through-Close'] = through.Lvl - df.Close[current]
        df.loc[current, 'through-Open'] = through.Lvl - df.Open[current]
        df.loc[current, 'through-Min'] = through.Lvl - Min
        df.loc[current, 'through-Max'] = through.Lvl - Max
        for val, column in zip(through[2:], through.index[2:]):
            df.loc[current, f'through_{column}'] = val
    else:
        df.loc[current, 'no_through'] = 1

   distances = ["High-below","Low-below","Close-below","Open-below","Min-below","Max-below",
                "above-High","above-Low","above-Close","above-Open","above-Min","above-Max",
                "through-High","through-Low","through-Close","through-Open",'through-Max','through-Min']
   for d in distances:
     df[f"Relative-{d}"] = df[d] / df["avg_candle"]

  #  for column in ["below","above","through"]:
  #     mask = (df[f'{column}_last_MReversal'] == 0) & (df[f'{column}_last_MReversal'].shift(1) != 0)
  #     mask.iloc[0] = False
  #     df.loc[mask, f'{column}_last_MReversal'] = df.loc[:,f'below_last_MReversal'].shift(1).loc[mask] + 1


# Functions

In [57]:
def prepare_data(data_file,start_year = None):
  df = pd.read_csv(data_file)
  if "Ticker" in df.columns:
    df = df.drop(columns = ["Ticker"])
  if "Adj_Close" in df.columns:
    df = df.drop(columns = ["Adj_Close"])
  for column in df.columns:
    if column.lower() == "date":
      df[column] = pd.to_datetime(df[column])
      df.set_index(column, inplace=True)
  if start_year != None:
     df = df[df.index.year > start_year-1]
  high_points = df.High
  low_points = df.Low
  mins, maxs,r,r = get_reversal(high_points)
  max_points = high_points.loc[high_points.index[maxs]]
  max_mins, max_maxs,r,r = get_reversal(max_points)
  max_maxs = [maxs[i] for i in max_maxs]
  min_mins = get_mins(low_points,max_maxs)
  add_reversal(df,high_points,min_mins, max_maxs)
  add_major_trend(df,min_mins,max_maxs)
  add_minor_trend(df,mins,maxs)
  add_trend_age(df,min_mins,max_maxs)
  df["avg_candle"] = df['High'].rolling('7D').mean() - df['Low'].rolling('7D').mean()
  add_SR(df)
  return df
def split_data(df,lookback):
  target = "Reversal"
  drop = ["next_Reversal","Reversal","Reversal_rate","Major_trend"]
  train, test = df.iloc[:-lookback], df.iloc[-lookback:]
  X_train = train.drop(columns = drop)
  y_train = train[target]
  X_test = test.drop(columns = drop)
  y_test = test[target]
  dtrain = xgb.DMatrix(X_train, label=y_train)
  dtest = xgb.DMatrix(X_test, label=y_test)
  return dtrain, dtest
def train_model(df, dtrain):
  params = {'max_depth': 1000,'eta': 0.1,'gamma': 0.8,'min_child_weight': 0.3,'subsample': 0.9,'colsample_bytree': 0.9,
          'objective': 'multi:softmax','seed': 42,'num_class':3}
  model = xgb.train(params, dtrain, 1000)
  return model
def plot(df,test,support_resistance = False):
  fig = go.Figure()
  fig = go.Figure(data=[go.Candlestick(x=test.index,
                  open=test['Open'],
                  high=test['High'],
                  low=test['Low'],
                  close=test['Close'])])
  fig.update_layout(xaxis_title='Date')

  if support_resistance == True:
    high_points = df.High
    low_points = df.Low
    mins, maxs,r,r = get_reversal(high_points)
    max_points = high_points.loc[high_points.index[maxs]]
    max_mins, max_maxs,r,r = get_reversal(max_points)
    max_maxs = [maxs[i] for i in max_maxs]
    min_mins = get_mins(low_points,max_maxs)

    lvls = get_levels(max_maxs,min_mins,high_points,low_points,df)
    lvls["Start"][lvls.Start < test.index[0]] = test.index[0]
    lvls = lvls[(lvls.Value > test.Low.min()) & (lvls.Value < test.High.max())]
    lvls = lvls[lvls.Start < test.index[-1]]
    for index, row in lvls.iterrows():
      fig.add_trace(go.Scatter(x=[row["Start"],test.index[-1]], y=[row["Value"],row["Value"]], mode='lines'))


  avg_candle_length = test["High"] - test["Low"]
  avg_arrow_length = avg_candle_length.mean()

  for i in range(len(test["Prediction"])):
      if test["Prediction"][i] != 0:
          arrow_color = "green" if test["Prediction"][i] == 1 else "red"
          ay = 30 if test["Prediction"][i] == 1 else -30
          y = test.Low[i] if test["Prediction"][i] == 1 else test.High[i]
          fig.add_annotation(x=test.index[i], y=y, ax=0, ay=ay,arrowwidth=2,
                            showarrow=True, arrowhead=1, arrowcolor=arrow_color)
  fig.show()

# Predict

In [80]:
data_file = "CBQK_EOD.csv"
start_year = 2014
plot_support_resistance = False
lookback = 300


# df = prepare_data(data_file,start_year = start_year)
# dtrain, dtest = split_data(df,lookback)
# model = train_model(df,dtrain)
# test = df.iloc[-lookback:]
# y_pred = model.predict(dtest)
# test["Prediction"] = y_pred.astype(int)
plot(df,test,support_resistance = plot_support_resistance)

In [ ]:
import os
df = pd.DataFrame()
for filename in os.listdir("/content"):
    f = os.path.join("/content", filename)
    if os.path.isfile(f):
        df = pd.concat([df,prepare_data(f,start_year = 2015)])